In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import string

print("modules imported successfully.")

modules imported successfully.


In [2]:
data = pd.read_csv("spam.csv", encoding = 'latin1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data = data.rename(columns ={"v1":"Category", "v2":"Message"})
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# def clean_text(Message):
#     '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
#     and remove words containing numbers.'''
#     Message = Message.lower()
#     Message = re.sub('\[.*?\]', '', Message)
#     Message = re.sub('https?://\S+|www\.\S+', '', Message)
#     Message = re.sub('<.*?>+', '', Message)
#     Message = re.sub('[%s]' % re.escape(string.punctuation), '', Message)
#     Message = re.sub('\n', '', Message)
#     Message = re.sub('\w*\d\w*', '', Message)
#     # Message = Message.split()
#     # ps = PorterStemmer()
#     # Message = [ps.stem(word) for word in Message if not word in set(stopwords.words('english'))]
#     # Message = ' '.join(Message)
#     return Message

# data['Message']=data['Message'].apply(clean_text)

# data.head()

In [ ]:
x=data['Message']
y=data['Category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)